In [1]:
import os
import numpy as np
import scipy as sc
import pandas as pd
import seaborn as sns
from pathlib import Path
import glob
import mdtraj as md
from scipy.spatial import distance
import itertools
import matplotlib.pyplot as plt
from sklearn import manifold
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list, fcluster,leaders
import multiprocessing
from multiprocessing import Pool
from concurrent.futures import ProcessPoolExecutor
import random
import palettable
from tqdm import tqdm

# Set multiprocessing start method for macOS compatibility
# Try to use 'fork' if available, otherwise use 'spawn'
try:
    multiprocessing.set_start_method('fork', force=True)
except RuntimeError:
    # If fork is not available, use spawn (default on macOS)
    try:
        multiprocessing.set_start_method('spawn', force=True)
    except RuntimeError:
        pass  # Already set, continue

ModuleNotFoundError: No module named 'mdtraj'

In [ ]:
# def compute_Q(XYZ_r,XYZ_t,length,sigma=1.0):
#     return (np.divide(np.sum(np.exp(-(np.power(distance.cdist(XYZ_r,XYZ_r, 'euclidean')-distance.cdist(XYZ_t,XYZ_t, 'euclidean') ,2))/(2.0*np.power(sigma,2)))),length*(length)))

In [ ]:
## Q - Wolynes

def compute_Q(XYZ_r, XYZ_t, length, N, M, sigma=1.0):
    d1 = distance.cdist(XYZ_r, XYZ_r, 'euclidean')
    d2 = distance.cdist(XYZ_t, XYZ_t, 'euclidean')

    # Set the diagonal elements up to N diags with np.inf (close neighbors)
    for i in range(N):
        if i < length:
            np.fill_diagonal(d1[:, i:], np.inf)
            np.fill_diagonal(d1[i:, :], np.inf)
            np.fill_diagonal(d2[:, i:], np.inf)
            np.fill_diagonal(d2[i:, :], np.inf)
    
    # Set the diagonal elements beyond M diags with np.inf (long-range neighbors)
    for i in range(M, length):
        np.fill_diagonal(d1[:, i:], np.inf)
        np.fill_diagonal(d1[i:, :], np.inf)
        np.fill_diagonal(d2[:, i:], np.inf)
        np.fill_diagonal(d2[i:, :], np.inf)

    # Replace nan values with 0
    d1 = np.nan_to_num(d1, posinf=0)
    d2 = np.nan_to_num(d2, posinf=0)

    return np.divide(np.sum(np.exp(-(np.power(d1 - d2, 2)) / (2.0 * np.power(sigma, 2)))), length * length)


# def compute_Q(XYZ_r, XYZ_t, length, N, M, sigma=1.0):
#     # Compute pairwise euclidean distances
#     d1 = distance.cdist(XYZ_r, XYZ_r, 'euclidean')
#     d2 = distance.cdist(XYZ_t, XYZ_t, 'euclidean')

#     # Mask close neighbors and long-range neighbors
#     for i in range(length):
#         if i < N or i >= M:
#             d1[i:, i] = np.inf
#             d1[i, i:] = np.inf
#             d2[i:, i] = np.inf
#             d2[i, i:] = np.inf

#     # Replace np.inf with 0 for subsequent calculations
#     d1 = np.nan_to_num(d1, posinf=0)
#     d2 = np.nan_to_num(d2, posinf=0)

#     # Calculate the Q metric
#     diff_squared = np.power(d1 - d2, 2)
#     return np.sum(np.exp(-diff_squared / (2 * sigma ** 2))) / (length ** 2)


In [ ]:
## Q - Onuchic
# def compute_Q(XYZ_r, XYZ_t, length, N, M,  sigma=2.0):

#     d1 = distance.cdist(XYZ_r, XYZ_r, 'euclidean')
#     d2 = distance.cdist(XYZ_t, XYZ_t, 'euclidean')
    
#     # Ignore close neighbors up to N diags by setting them to a large value
#     for i in range(N):
#         if i < length:
#             np.fill_diagonal(d1[:, i:], float('inf'))
#             np.fill_diagonal(d1[i:, :], float('inf'))
#             np.fill_diagonal(d2[:, i:], float('inf'))
#             np.fill_diagonal(d2[i:, :], float('inf'))
    
#     # Ignore long-range neighbors beyond M diags by setting them to a large value
#     for i in range(M, length):
#         np.fill_diagonal(d1[:, i:], float('inf'))
#         np.fill_diagonal(d1[i:, :], float('inf'))
#         np.fill_diagonal(d2[:, i:], float('inf'))
#         np.fill_diagonal(d2[i:, :], float('inf'))

#     # Calculate binary matrix of contacts
#     contacts = np.abs(d1 - d2) < sigma
#     return contacts.sum() / (length * length)


In [ ]:
def load_pdb_models(pdb_path):
    """
    Load all models from a PDB file.
    Returns a list of numpy arrays, each representing one model's coordinates (N, 3).
    """
    models = []
    current_coords = []
    in_model = False
    
    with open(pdb_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            rec = line[:6].strip().upper()
            
            if rec == "MODEL":
                # If nested MODEL found without ENDMDL, flush previous if any
                if in_model and current_coords:
                    models.append(np.array(current_coords, dtype=np.float64))
                    current_coords = []
                in_model = True
                continue
                
            if rec in ("ATOM", "HETATM") and in_model:
                try:
                    # PDB columns: x[30:38], y[38:46], z[46:54]
                    x = float(line[30:38])
                    y = float(line[38:46])
                    z = float(line[46:54])
                    current_coords.append([x, y, z])
                except (ValueError, IndexError):
                    continue
                    
            elif rec == "ENDMDL" and in_model:
                if current_coords:
                    models.append(np.array(current_coords, dtype=np.float64))
                    current_coords = []
                in_model = False
    
    # Handle files that end without ENDMDL
    if in_model and current_coords:
        models.append(np.array(current_coords, dtype=np.float64))
    
    return models


def get_XYZ(files, file_type='auto'):
    """
    Load coordinates from files (supports both PDB and GRO formats).
    
    Parameters:
    -----------
    files : list
        List of file paths
    file_type : str
        'auto' (detect from extension), 'pdb', or 'gro'
    
    Returns:
    --------
    List of coordinate arrays. For multi-model PDB files, expands to return 
    one array per model (so if file has 100 models, returns 100 arrays).
    """
    all_xyz = []
    
    for file in files:
        # Determine file type
        if file_type == 'auto':
            ext = os.path.splitext(file)[1].lower()
            is_pdb = ext == '.pdb'
        else:
            is_pdb = file_type.lower() == 'pdb'
        
        if is_pdb:
            # Load all models from PDB file
            models = load_pdb_models(file)
            # If file has multiple models, add each as a separate structure
            if len(models) > 0:
                all_xyz.extend(models)
            else:
                print(f"Warning: No models found in {file}")
        else:
            # Use MDTraj for GRO files (or other formats)
            try:
                traj = md.load(file)
                if traj.n_frames > 0:
                    # For GRO files, use first frame
                    all_xyz.append(traj.xyz[0])
            except Exception as e:
                print(f"Warning: Could not load {file} with MDTraj: {e}")
    
    return all_xyz

In [ ]:
def calculate_reduced_structure(XYZ,patches):

    current_patch_type = patches[0]
    patch_indices = []

    # This list will store the new reduced polymer structure
    reduced_structure = []
    reduced_sequence = []
    for i, patch in enumerate(patches):
        if patch == current_patch_type:
            patch_indices.append(i)
        if patch != current_patch_type or i == len(patches) - 1:  # Adding condition for the last element
            # When the patch changes or we are at the end, calculate the COM of the current patch
            COM = np.mean(XYZ[patch_indices, :], axis=0)
            reduced_structure.append(COM)
            reduced_sequence.append(current_patch_type)
            
            patch_indices = [i]
            current_patch_type = patch

    reduced_structure = np.array(reduced_structure)
    return reduced_structure, reduced_sequence

In [ ]:
def compute_reduced_traj(XYZ_t,patches):
    XYZ_t_reduced = []
    for i in range(len(XYZ_t)):
        m,a = calculate_reduced_structure(XYZ_t[i],patches)
        XYZ_t_reduced.append(m)
    return XYZ_t_reduced

In [ ]:
def compute_Q_parallel_all(args):
    i, j, XYZ_MiChroM, XYZ_Full, XYZ_HP,beads,neig_short,neig_long,sigma = args
    Q_M_M = compute_Q(XYZ_MiChroM[i], XYZ_MiChroM[j], beads,neig_short,neig_long,sigma)
    Q_F_F = compute_Q(XYZ_Full[i], XYZ_Full[j], beads,neig_short,neig_long,sigma)
    Q_M_F = compute_Q(XYZ_MiChroM[i], XYZ_Full[j], beads,neig_short,neig_long,sigma)
    Q_H_H = compute_Q(XYZ_HP[i], XYZ_HP[j], beads,neig_short,neig_long,sigma)
    Q_M_H = compute_Q(XYZ_MiChroM[i], XYZ_HP[j], beads,neig_short,neig_long,sigma)
    Q_F_H = compute_Q(XYZ_Full[i], XYZ_HP[j], beads,neig_short,neig_long,sigma)
    
    return Q_M_M, Q_F_F, Q_M_F, Q_H_H, Q_M_H, Q_F_H

In [ ]:
def compute_Q_parallel(args):
    i, j, XYZ_1,XYZ_2,beads = args
    return compute_Q(XYZ_1[i], XYZ_2[j], beads)

In [ ]:
def upper_triangle_to_square(arr):
    """
    Convert a 1D array representing the upper triangle of a matrix 
    (including the diagonal) into a square matrix.
    """
    n = int((-1 + np.sqrt(1 + 8 * len(arr))) // 2)
    matrix = np.zeros((n, n))
    matrix[np.triu_indices(n)] = arr
    matrix += matrix.T - np.diag(matrix.diagonal())
    return matrix

In [ ]:
def array_to_square(arr):
    """
    Convert a 1D array to a square matrix.
    """
    n = int(np.sqrt(len(arr)))
    if n*n != len(arr):
        raise ValueError("Input array length is not a perfect square")
    return arr.reshape(n, n)

In [ ]:
def compute_RG(xyz):
    R"""
    Calculates the Radius of Gyration of a chromosome chain.
    
    Returns:
            Returns the Radius of Gyration in units of :math:`\sigma`
    """
    data = xyz
    data = data - np.mean(data, axis=0)[None,:]
    return np.sqrt(np.sum(np.var(np.array(data), 0)))

In [ ]:
def select_random_gro_files(base_dir, X, seed=None):
    """
    Select X random .gro files from the directory structure.

    :param base_dir: The base directory where the folders are.
    :param X: Number of random files to select.
    :param seed: Seed for reproducibility.
    :return: List of randomly selected .gro files.
    """
    # Set the random seed for reproducibility
    if seed is not None:
        random.seed(seed)

    # Search for all .gro files in the numbered directories
    all_files = glob.glob(f"{base_dir}/*/*.gro")
    # Exclude files named run_M_0_block0.gro
    files_MiChroM = [f for f in all_files if not f.endswith('run_M_0_block0.gro')]
    # Randomly select X files
    selected_files = random.sample(files_MiChroM, X)

    return selected_files

In [ ]:
def coarsen(reduction, x, axes, trim_excess=False):
    """
    Coarsen an array by applying reduction to fixed size neighborhoods.
    Adapted from `dask.array.coarsen` to work on regular numpy arrays.

    Parameters
    ----------
    reduction : function
        Function like np.sum, np.mean, etc...
    x : np.ndarray
        Array to be coarsened
    axes : dict
        Mapping of axis to coarsening factor
    trim_excess : bool, optional
        Remove excess elements. Default is False.

    Examples
    --------
    Provide dictionary of scale per dimension

    >>> x = np.array([1, 2, 3, 4, 5, 6])
    >>> coarsen(np.sum, x, {0: 2})
    array([ 3,  7, 11])

    >>> coarsen(np.max, x, {0: 3})
    array([3, 6])

    >>> x = np.arange(24).reshape((4, 6))
    >>> x
    array([[ 0,  1,  2,  3,  4,  5],
           [ 6,  7,  8,  9, 10, 11],
           [12, 13, 14, 15, 16, 17],
           [18, 19, 20, 21, 22, 23]])

    >>> coarsen(np.min, x, {0: 2, 1: 3})
    array([[ 0,  3],
           [12, 15]])

    See also
    --------
    dask.array.coarsen

    """
    # Insert singleton dimensions if they don't exist already
    for i in range(x.ndim):
        if i not in axes:
            axes[i] = 1

    if trim_excess:
        ind = tuple(
            slice(0, -(d % axes[i])) if d % axes[i] else slice(None, None)
            for i, d in enumerate(x.shape)
        )
        x = x[ind]

    # (10, 10) -> (5, 2, 5, 2)
    newdims = [(x.shape[i] // axes[i], axes[i]) for i in range(x.ndim)]
    newshape = tuple(np.concatenate(newdims))
    reduction_axes = tuple(range(1, x.ndim * 2, 2))
    return reduction(x.reshape(newshape), axis=reduction_axes)

In [ ]:
# Load PDB files directly
# You can modify these paths to point to your PDB files
pdb_file_1 = "/Users/amk19/Desktop/ChromatinVAE/outputs_cluster/Generated_Samples/generated_samples_uniform.pdb"
pdb_file_2 = "/Users/amk19/Desktop/ChromatinVAE/Data/chromosome21_aligned.pdb"

# Convert single PDB files to lists (get_XYZ expects a list of files)
# For comparison, we'll treat each model in the PDB as a separate structure
files_MiChroM = [pdb_file_1]  # Generated samples
files_Full = [pdb_file_2]      # Reference data

# If you want to use only a subset of models, you can load separately
# For now, using all models from each file
files_HP = []  # Not used in this comparison, but keeping for compatibility

In [ ]:
# files_MiChroM = glob.glob("/mnt/d/Mammoth/Simulations/random_M_Run_M/*.gro")
# # files_Full = glob.glob("/mnt/d/Mammoth/Simulations/random_M_Run_53/*.gro")
# files_HP = glob.glob("/mnt/d/Mammoth/Simulations/random_M_Run_HP/*.gro")

In [ ]:
len(files_MiChroM),len(files_Full),len(files_HP)

(1, 1, 0)

In [ ]:
# Q_M_M=[]
# Q_F_F=[]
# Q_H_H=[]
# Q_M_F=[]
# Q_M_H=[]
# Q_F_H=[]

# for i,j in itertools.combinations_with_replacement(range(0,len(files_MiChroM)),2):
#     print(i,j)
#     Q_M_M.append(compute_Q(XYZ_MiChroM[i],XYZ_MiChroM[j],481,neig_short,neig_long,sigma))
#     Q_F_F.append(compute_Q(XYZ_Full[i],XYZ_Full[j],481,neig_short,neig_long,sigma))
#     Q_M_F.append(compute_Q(XYZ_MiChroM[i],XYZ_Full[j],481,neig_short,neig_long,sigma))
#     Q_H_H.append(compute_Q(XYZ_HP[i],XYZ_HP[j],481,neig_short,neig_long,sigma))
#     Q_M_H.append(compute_Q(XYZ_MiChroM[i],XYZ_HP[j],481,neig_short,neig_long,sigma))
#     Q_F_H.append(compute_Q(XYZ_Full[i],XYZ_HP[j],481,neig_short,neig_long,sigma))


In [ ]:
# Load coordinates from PDB files
# Each model in a PDB file will be loaded as a separate structure
XYZ_MiChroM_all = get_XYZ(files_MiChroM, file_type='pdb')
XYZ_Full_all = get_XYZ(files_Full, file_type='pdb')

# Check if we have HP files, otherwise use empty list
if len(files_HP) > 0:
    XYZ_HP_all = get_XYZ(files_HP, file_type='pdb')
else:
    XYZ_HP_all = []

print(f"Loaded {len(XYZ_MiChroM_all)} structures from generated samples")
print(f"Loaded {len(XYZ_Full_all)} structures from reference data")
if len(XYZ_HP_all) > 0:
    print(f"Loaded {len(XYZ_HP_all)} structures from HP data")
    
# Check structure sizes
if len(XYZ_MiChroM_all) > 0:
    print(f"Generated samples have {len(XYZ_MiChroM_all[0])} beads per structure")
if len(XYZ_Full_all) > 0:
    print(f"Reference data has {len(XYZ_Full_all[0])} beads per structure")

Loaded 1000 structures from generated samples
Loaded 7591 structures from reference data
Generated samples have 651 beads per structure
Reference data has 651 beads per structure


In [ ]:
# Optional: Load patch information for structure reduction
# If you don't have a patch file, you can skip this cell and use the full structures
# Comment out the section below if you want to use full structures without reduction

use_patch_reduction = False  # Set to True if you have a patch file

if use_patch_reduction:
    filename = "/mnt/d/Mammoth/Simulations/M_Run_M/M_for_Training/chr_HiC_scaffold_23_250kb_eigen.seq"
    # Lists to store indices and patches
    indices = []
    patches = []
    with open(filename, 'r') as file:
        for line in file:
            # Split each line into index and patch
            index, patch = line.split()
            indices.append(int(index))
            patches.append(patch)
else:
    patches = None
    print("Using full structures without patch reduction")

Using full structures without patch reduction


In [ ]:
# Only calculate reduced structure if patches are available
if patches is not None:
    reduced_structure,reduced_sequence = calculate_reduced_structure(XYZ_MiChroM_all[0],patches)
    print(f"Reduced structure shape: {reduced_structure.shape}, Reduced sequence length: {len(reduced_sequence)}")
else:
    print("Skipping reduced structure calculation - using full structures (patches is None)")
    reduced_structure = None
    reduced_sequence = None

Skipping reduced structure calculation - using full structures (patches is None)


In [ ]:
# Only print reduced structure info if it was calculated
if reduced_structure is not None:
    print(f"Reduced structure shape: {np.shape(reduced_structure)}, Reduced sequence length: {len(reduced_sequence)}")
else:
    print("Full structure shape:", np.shape(XYZ_MiChroM_all[0]))

Full structure shape: (651, 3)


In [ ]:
## Apply patch reduction if patches are available, otherwise use full structures

if patches is not None:
    XYZ_MiChroM = compute_reduced_traj(XYZ_MiChroM_all, patches)
    XYZ_Full = compute_reduced_traj(XYZ_Full_all, patches)
    if len(XYZ_HP_all) > 0:
        XYZ_HP = compute_reduced_traj(XYZ_HP_all, patches)
    else:
        XYZ_HP = []
else:
    # Use full structures without reduction
    XYZ_MiChroM = XYZ_MiChroM_all
    XYZ_Full = XYZ_Full_all
    XYZ_HP = XYZ_HP_all
    
print(f"MiChroM: {len(XYZ_MiChroM)} structures")
print(f"Full: {len(XYZ_Full)} structures")
print(f"HP: {len(XYZ_HP)} structures")

MiChroM: 1000 structures
Full: 7591 structures
HP: 0 structures


In [ ]:
# np.shape(XYZ_HP),np.shape(XYZ_HP_all)

In [ ]:
# ###All points

# XYZ_MiChroM = XYZ_MiChroM_all
# XYZ_Full = XYZ_Full_all
# XYZ_HP = XYZ_HP_all

In [ ]:
# fig = plt.figure(figsize=(12, 6))

# # Plot for averaged_structure
# ax1 = fig.add_subplot(121, projection='3d')
# ax1.scatter(XYZ_MiChroM[0][:, 0], XYZ_MiChroM[0][:, 1], XYZ_MiChroM[0][:, 2], marker='o')
# ax1.set_title("Averaged Structure")
# ax1.set_xlabel("X")
# ax1.set_ylabel("Y")
# ax1.set_zlabel("Z")

# # Plot for reduced_structure
# ax2 = fig.add_subplot(122, projection='3d')
# ax2.scatter(reduced_structure[:, 0], reduced_structure[:, 1], reduced_structure[:, 2], marker='o')
# ax2.set_title("Reduced Structure")
# ax2.set_xlabel("X")
# ax2.set_ylabel("Y")
# ax2.set_zlabel("Z")

# plt.show()


In [ ]:
# Determine number of beads (should be same for all structures in each group)
if len(XYZ_MiChroM) > 0:
    beads = len(XYZ_MiChroM[0])
else:
    beads = len(XYZ_Full[0]) if len(XYZ_Full) > 0 else 0

sigma = 2
neig_short = 5
neig_long = 200

print(f"Number of beads: {beads}")
print(f"Q parameters: sigma={sigma}, neig_short={neig_short}, neig_long={neig_long}")

Number of beads: 651
Q parameters: sigma=2, neig_short=5, neig_long=200


In [ ]:
# # n_cores=12
# # pool = Pool(processes=n_cores)  #i, j, XYZ_MiChroM, XYZ_Full, XYZ_HP,beads,sigma,N = args
# # results = pool.map(compute_Q_parallel_all, [(i, j, XYZ_MiChroM, XYZ_Full, XYZ_HP, beads,neig_short,neig_long,sigma) for i, j in itertools.combinations_with_replacement(range(len(files_MiChroM)), 2)])

# n_cores = 12
# pool = Pool(processes=n_cores)

# # Combinations for cross dataset comparisons
# cross_dataset_combinations = list(itertools.product(range(len(files_MiChroM)), repeat=2))



In [ ]:
# Global variables for worker functions (set before multiprocessing)
_worker_XYZ_MiChroM = None
_worker_XYZ_Full = None
_worker_XYZ_HP = None
_worker_beads = None
_worker_neig_short = None
_worker_neig_long = None
_worker_sigma = None

def init_worker(XYZ_MiChroM, XYZ_Full, XYZ_HP, beads, neig_short, neig_long, sigma):
    """Initialize worker process with shared data"""
    global _worker_XYZ_MiChroM, _worker_XYZ_Full, _worker_XYZ_HP
    global _worker_beads, _worker_neig_short, _worker_neig_long, _worker_sigma
    _worker_XYZ_MiChroM = XYZ_MiChroM
    _worker_XYZ_Full = XYZ_Full
    _worker_XYZ_HP = XYZ_HP
    _worker_beads = beads
    _worker_neig_short = neig_short
    _worker_neig_long = neig_long
    _worker_sigma = sigma

def compute_Q_parallel_same(args):
    """Compute Q for same-dataset comparisons - takes only indices"""
    i, j = args
    Q_M_M = compute_Q(_worker_XYZ_MiChroM[i], _worker_XYZ_MiChroM[j], 
                      _worker_beads, _worker_neig_short, _worker_neig_long, _worker_sigma)
    Q_F_F = compute_Q(_worker_XYZ_Full[i], _worker_XYZ_Full[j], 
                      _worker_beads, _worker_neig_short, _worker_neig_long, _worker_sigma)
    
    # Only compute HP comparisons if HP dataset exists and indices are valid
    if len(_worker_XYZ_HP) > 0 and i < len(_worker_XYZ_HP) and j < len(_worker_XYZ_HP):
        Q_H_H = compute_Q(_worker_XYZ_HP[i], _worker_XYZ_HP[j], 
                         _worker_beads, _worker_neig_short, _worker_neig_long, _worker_sigma)
    else:
        Q_H_H = np.nan
    
    return Q_M_M, Q_F_F, Q_H_H

In [ ]:
def compute_Q_parallel_cross(args):
    """Compute Q for cross-dataset comparisons - takes only indices"""
    i, j = args
    Q_M_F = compute_Q(_worker_XYZ_MiChroM[i], _worker_XYZ_Full[j], 
                     _worker_beads, _worker_neig_short, _worker_neig_long, _worker_sigma)
    
    # Only compute HP comparisons if HP dataset exists and indices are valid
    if len(_worker_XYZ_HP) > 0 and i < len(_worker_XYZ_HP) and j < len(_worker_XYZ_HP):
        Q_M_H = compute_Q(_worker_XYZ_MiChroM[i], _worker_XYZ_HP[j], 
                         _worker_beads, _worker_neig_short, _worker_neig_long, _worker_sigma)
        Q_F_H = compute_Q(_worker_XYZ_Full[i], _worker_XYZ_HP[j], 
                         _worker_beads, _worker_neig_short, _worker_neig_long, _worker_sigma)
    else:
        Q_M_H = np.nan
        Q_F_H = np.nan
    
    return Q_M_F, Q_M_H, Q_F_H

In [ ]:
def upper_triangle_to_square_without_diagonal(arr):
    n = int(np.sqrt(2*len(arr) + 0.25) - 0.5)  # New formula to calculate size
    matrix = np.zeros((n, n))
    indices = np.triu_indices(n, 1)  # Exclude diagonal
    matrix[indices] = arr
    matrix += matrix.T
    return matrix

In [ ]:
n_cores=60

# Calculate Q values - using actual number of structures loaded (not number of files)
n_structures_M = len(XYZ_MiChroM)
n_structures_F = len(XYZ_Full)
n_structures_H = len(XYZ_HP)

print(f"Computing Q values for {n_structures_M} MiChroM, {n_structures_F} Full, {n_structures_H} HP structures")

# Prepare tasks - now ONLY indices (much faster and smaller memory footprint)
# For same-dataset comparisons (within each dataset)
same_tasks = [(i, j) for i, j in itertools.combinations_with_replacement(range(n_structures_M), 2)]

# For cross-dataset comparisons (between datasets)
if n_structures_H > 0:
    cross_combinations = list(itertools.product(range(min(n_structures_M, n_structures_F, n_structures_H)), repeat=2))
else:
    cross_combinations = list(itertools.product(range(min(n_structures_M, n_structures_F)), repeat=2))

cross_tasks = [(i, j) for i, j in cross_combinations]

print(f"Created {len(same_tasks)} same-dataset tasks and {len(cross_tasks)} cross-dataset tasks")
print(f"Optimized: tasks now only contain indices (much faster!)")

# For multiprocessing on macOS with spawn, use initializer to pass arrays once
try:
    # Create Pool with initializer to pass arrays only once per worker
    print("Initializing worker processes with shared data...")
    pool = Pool(processes=n_cores, 
                initializer=init_worker, 
                initargs=(XYZ_MiChroM, XYZ_Full, XYZ_HP, beads, neig_short, neig_long, sigma))
    
    # Use imap for progress tracking with multiprocessing
    print("Computing same-dataset comparisons...")
    results_same = list(tqdm(pool.imap(compute_Q_parallel_same, same_tasks, chunksize=10), 
                            total=len(same_tasks), 
                            desc="Same-dataset Q values"))
    
    print("Computing cross-dataset comparisons...")
    results_cross = list(tqdm(pool.imap(compute_Q_parallel_cross, cross_tasks, chunksize=10), 
                             total=len(cross_tasks), 
                             desc="Cross-dataset Q values"))
    
    pool.close()
    pool.join()
    print("Completed parallel Q calculations using multiprocessing")
    
except Exception as e:
    print(f"Multiprocessing failed: {e}")
    print("Falling back to serial computation (this will be slower)...")
    
    # Serial fallback - initialize worker globals first
    init_worker(XYZ_MiChroM, XYZ_Full, XYZ_HP, beads, neig_short, neig_long, sigma)
    
    # Serial computation with progress bars
    print(f"Processing {len(same_tasks)} same-dataset comparisons...")
    results_same = [compute_Q_parallel_same(task) for task in tqdm(same_tasks, desc="Same-dataset Q values")]
    
    print(f"Processing {len(cross_tasks)} cross-dataset comparisons...")
    results_cross = [compute_Q_parallel_cross(task) for task in tqdm(cross_tasks, desc="Cross-dataset Q values")]
    
    print("Completed Q calculations using serial computation")




Computing Q values for 1000 MiChroM, 7591 Full, 0 HP structures
Created 500500 same-dataset tasks and 1000000 cross-dataset tasks
Optimized: tasks now only contain indices (much faster!)
Initializing worker processes with shared data...
Computing same-dataset comparisons...


Same-dataset Q values:  34%|███▍      | 170013/500500 [05:43<12:06, 454.61it/s]

In [ ]:
# Unpack results
Q_M_M, Q_F_F, Q_H_H = zip(*results_same)
Q_M_F, Q_M_H, Q_F_H = zip(*results_cross)

# Convert to arrays and handle NaN values if HP dataset is empty
Q_M_M = np.array(Q_M_M)
Q_F_F = np.array(Q_F_F)
Q_H_H = np.array(Q_H_H)
Q_M_F = np.array(Q_M_F)
Q_M_H = np.array(Q_M_H)
Q_F_H = np.array(Q_F_H)

# Filter out NaN values if HP dataset was empty
if len(XYZ_HP) == 0:
    print("Note: HP dataset is empty - filtering out NaN values")
    # Filter Q_H_H (same dataset comparisons)
    Q_H_H = Q_H_H[~np.isnan(Q_H_H)]
    # Filter Q_M_H and Q_F_H (cross dataset comparisons)
    Q_M_H = Q_M_H[~np.isnan(Q_M_H)]
    Q_F_H = Q_F_H[~np.isnan(Q_F_H)]

In [ ]:
# Rg_M_M=[]
# for i in range(len(XYZ_MiChroM)):
#     Rg_M_M.append(compute_RG(XYZ_MiChroM[i]))

# Rg_F_F=[]
# for i in range(len(XYZ_Full)):
#     Rg_F_F.append(compute_RG(XYZ_Full[i]))

# Rg_H_H=[]
# for i in range(len(XYZ_HP)):
#     Rg_H_H.append(compute_RG(XYZ_HP[i]))

In [ ]:
# np.array(Q_F_H) - Q_M_M_single

In [ ]:
# n_cores=5
# pool = Pool(processes=n_cores)
# Q_M_M = pool.map(compute_Q_parallel, [(i, j, XYZ_MiChroM, XYZ_MiChroM,beads) for i, j in itertools.combinations_with_replacement(range(len(files_MiChroM)), 2)])
# Q_M_H = pool.map(compute_Q_parallel, [(i, j, XYZ_MiChroM, XYZ_HP,beads) for i, j in itertools.combinations_with_replacement(range(len(files_MiChroM)), 2)])
# Q_H_H = pool.map(compute_Q_parallel, [(i, j, XYZ_HP, XYZ_HP,beads) for i, j in itertools.combinations_with_replacement(range(len(files_MiChroM)), 2)])

In [ ]:
# N=len(Q_M_F)
# l=int((-1+np.sqrt(1+8*N))/2)

# print(l,N)

In [ ]:
# M_Q_M_F= np.zeros((l,l))
# indices = np.triu_indices(l)
# M_Q_M_F[indices] = Q_M_F[0:N]

# matrix_all = M_Q_M_F + M_Q_M_F.T - np.diag(M_Q_M_F.diagonal())

In [ ]:
Q_M_M = np.array(Q_M_M)
Q_F_F = np.array(Q_F_F)
Q_H_H = np.array(Q_H_H)
Q_M_F = np.array(Q_M_F)
Q_M_H = np.array(Q_M_H)
Q_F_H = np.array(Q_F_H)

In [ ]:
len(Q_M_F),len(Q_M_M),len(Q_F_F),len(Q_H_H),len(Q_M_H),len(Q_F_H)

In [ ]:
Q_M_M_matrix = upper_triangle_to_square(Q_M_M)
Q_F_F_matrix = upper_triangle_to_square(Q_F_F)
Q_H_H_matrix = upper_triangle_to_square(Q_H_H)

Q_M_F_matrix = array_to_square(Q_M_F)
Q_M_H_matrix = array_to_square(Q_M_H)
Q_F_H_matrix = array_to_square(Q_F_H)


In [ ]:
merged_matrix = np.block([
    [Q_M_M_matrix, Q_M_F_matrix, Q_M_H_matrix],
    [Q_M_F_matrix.T, Q_F_F_matrix, Q_F_H_matrix],
    [Q_M_H_matrix.T, Q_F_H_matrix.T, Q_H_H_matrix]
])


In [ ]:
np.save('/home/vinicius/Work/Mammoth/Simulations/Q_Matrix.npy',merged_matrix)

In [ ]:
# merged_matrix = np.block([
#     [Q_M_M_matrix, Q_M_H_matrix],
#     [Q_M_H_matrix.T, Q_H_H_matrix]
# ])


In [ ]:
# matrix_removed = matrix_all[1:, 1:]

In [ ]:
# Q_M_M_matrix[8][8]

In [ ]:
# vmin=0
# vmax=0.1

# # plt.imshow(Q_M_M_matrix[0:-1][:,0:-1], interpolation='None', cmap='viridis_r', vmin=vmin, vmax=vmax)

# # plt.imshow(Q_F_F_matrix[0:-1][:,0:-1], interpolation='None', cmap='viridis_r', vmin=vmin, vmax=vmax)

# plt.imshow(Q_H_H_matrix[0:-1][:,0:-1], interpolation='None', cmap='viridis_r', vmin=vmin, vmax=vmax)

# plt.colorbar()
# plt.show()

In [ ]:
# cmap = sns.choose_cubehelix_palette()

In [ ]:
vmin=0.2
vmax=0.8
# palette = cubehelix.classic_16.mpl_colormap
# palette = palettable.mycarta.LinearL_4.mpl_colormap
# palette = palettable.lightbartlein.diverging.BlueDarkRed18_9.mpl_colormap
# palette = palettable.scientific.sequential.Acton_3_r.mpl_colormap

palette = palettable.cmocean.sequential.Ice_3_r.mpl_colormap

plt.imshow(merged_matrix, interpolation='None', cmap=palette, vmin=vmin, vmax=vmax)
# plt.savefig('/home/vinicius/Work/Mammoth/Simulations/Q_Matrix_test.jpg',dpi=600)
plt.show()

In [ ]:
dissimilarity_matrix = 1 - merged_matrix

In [ ]:
dissimilarity_matrix

In [ ]:
# # t=1.3 # sim 47
# # t = 4.2 # sim 1000
# # t= 1.4 # exp

# t=1.3

# Z = linkage(dissimilarity_matrix, method='complete')
# T = fcluster(Z, t=t, criterion='distance')

# plt.figure(figsize=(10, 5))
# d = dendrogram(Z, color_threshold=t)
# # d = dendrogram(Z)

# plt.savefig('/home/vinicius/Work/Mammoth/Simulations/Dendogram_test1.jpg',dpi=300)
# # plt.xticks([])
# # plt.savefig('/mnt/d/DT40/Centromere-Modeling/Data_Gro/Figures/Dendogram_3D_Images/Dendogram_sim_chr5_1000_set1.jpg',dpi=300)
# plt.show()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.cluster import hierarchy
# import seaborn as sns

# # Load precomputed dissimilarity data
# dissimilarity = dissimilarity_matrix

# # Compute linkage matrix using the dissimilarity data
# Z = hierarchy.linkage(dissimilarity, method='complete')

# # Compute optimal ordering of the rows and columns based on dendrogram linkage
# optimal_order = hierarchy.leaves_list(Z)

# # Reorder dissimilarity matrix and labels based on optimal order
# reordered_dissimilarity = dissimilarity[optimal_order, :]
# reordered_dissimilarity = reordered_dissimilarity[:, optimal_order]

# # Reordered labels
# reordered_labels = [i for i in optimal_order]

# # Plot dendrogram and matrix plot together
# fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 8), gridspec_kw={'width_ratios': [0.25, 0.75]})

# # Plot dendrogram on the left

# hierarchy.dendrogram(Z, ax=ax1, orientation='left', color_threshold=t)
# ax1.invert_yaxis()  # Reverse the y-axis of the dendrogram
# # ax1.tick_params(axis='y', labelsize=8)


# ax1.set_yticks([])
# ax1.set_yticklabels([])

# # Plot heatmap on the right
# sns.heatmap(reordered_dissimilarity, ax=ax2, cmap="YlGnBu", vmin=0,vmax=1, cbar=False)

# # ax2.set_xticks([i + 0.5 for i in range(len(reordered_labels))])
# # ax2.set_yticks([i + 0.5 for i in range(len(reordered_labels))])
# # ax2.set_xticklabels(reordered_labels, fontsize=8)
# # ax2.set_yticklabels(reordered_labels, fontsize=8, rotation=0)


# # Remove x-axis and y-axis ticks and labels
# ax2.set_xticks([])
# ax2.set_xticklabels([])
# ax2.set_yticks([])
# ax2.set_yticklabels([])



# # Add title and axis labels
# ax1.set_title("Dendrogram", fontsize=14)
# ax2.set_title("Dissimilarity Matrix", fontsize=14)
# ax2.set_xlabel("Structures", fontsize=12)
# ax2.set_ylabel("Structures", fontsize=12)

# # Adjust spacing between plots
# fig.tight_layout(pad=2.0)

# # Show plot

# # plt.savefig('/mnt/d/DT40/Centromere-Modeling/Data_Gro/Figures/Dendogram_3D_Images/Q_Matrix_Dendogram_sim_chr5_1000_set1.jpg',dpi=600)

# plt.show()


In [ ]:
# # Flatten the dissimilarity matrix to a 1D array
# flattened_matrix = dissimilarity_matrix.flatten()

# # Plotting the histogram using Seaborn
# sns.histplot(flattened_matrix, kde=False)
# plt.xlabel('Dissimilarity Value')
# plt.ylabel('Frequency')
# plt.title('Histogram of Dissimilarity Matrix')
# plt.grid(True)
# plt.show()

In [ ]:
len(Q_M_M),len(Q_F_F),len(Q_M_F)

In [ ]:
# # Flatten the matrix into a 1D array
# dissimilarity_data = dissimilarity_matrix.flatten()

# # Remove redundant elements if it's a symmetric matrix
# if np.allclose(dissimilarity_matrix, dissimilarity_matrix.T):
#     dissimilarity_data = dissimilarity_data[dissimilarity_data > 0]

# # Convert to a pandas Series
# dissimilarity_series = pd.Series(dissimilarity_data)

# # Create the density plot
# sns.kdeplot(dissimilarity_series,fill=True,)

# plt.xlabel('Dissimilarity Value')
# plt.ylabel('Frequency')
# plt.title('Histogram of Dissimilarity')
# plt.grid(True)



# data_save_folder = Path("/mnt/d/Mammoth/Images/Full-Inversion/")
# # file_plot = data_save_folder /  'histogram_Dissimilarity.jpeg'

# # plt.savefig(file_plot,bbox_inches='tight',dpi=600)


# plt.show()

In [ ]:
Q_M_M = np.array(Q_M_M)
Q_F_F = np.array(Q_F_F)
Q_H_H = np.array(Q_H_H)
Q_M_F = np.array(Q_M_F)
Q_M_H = np.array(Q_M_H)
Q_F_H = np.array(Q_F_H)

In [ ]:
len(Q_M_M[Q_M_M != 1.0]),len(Q_F_H)

In [ ]:
# data1 = {
#     'MiChroM_vs_MiChroM': Q_M_M[Q_M_M != 1.0],
#     'DirectInversion_vs_DirectInversion': Q_F_F[Q_F_F != 1.0],
#     'Homopolymer_vs_Homopolymer':Q_H_H[Q_H_H != 1.0] ,
# }

# data2 = {
#     'MiChroM_vs_DirectInversion':Q_M_F[Q_M_F != 1.0],
#     'MiChroM_vs_Homopolymer': Q_M_H[Q_M_H != 1.0],
#     'DirectInversion_vs_Homopolymer': Q_F_H[Q_F_H != 1.0],
# }


# df1 = pd.DataFrame(data1)
# df2 = pd.DataFrame(data2)

In [ ]:
# df_M_M = pd.DataFrame({'Q': Q_M_M[Q_M_M != 1.0], 'Type': 'MiChroM_vs_MiChroM'})
# df_F_F = pd.DataFrame({'Q': Q_F_F[Q_F_F != 1.0], 'Type': 'DirectInversion_vs_DirectInversion'})
# df_H_H = pd.DataFrame({'Q': Q_H_H[Q_H_H != 1.0], 'Type': 'Homopolymer_vs_Homopolymer'})
# df_M_F = pd.DataFrame({'Q': Q_M_F[Q_M_F != 1.0], 'Type': 'MiChroM_vs_DirectInversion'})
# df_M_H = pd.DataFrame({'Q': Q_M_H[Q_M_H != 1.0], 'Type': 'MiChroM_vs_Homopolymer'})
# df_F_H = pd.DataFrame({'Q': Q_F_H[Q_F_H != 1.0], 'Type': 'DirectInversion_vs_Homopolymer'})


# Create DataFrames, only including non-empty arrays
dataframes_list = []

if len(Q_M_M) > 0:
    df_M_M = pd.DataFrame({'Q': Q_M_M, 'Type': 'MiChroM_vs_MiChroM'})
    dataframes_list.append(df_M_M)
else:
    df_M_M = pd.DataFrame({'Q': [], 'Type': []})

if len(Q_F_F) > 0:
    df_F_F = pd.DataFrame({'Q': Q_F_F, 'Type': 'DirectInversion_vs_DirectInversion'})
    dataframes_list.append(df_F_F)
else:
    df_F_F = pd.DataFrame({'Q': [], 'Type': []})

if len(Q_H_H) > 0:
    df_H_H = pd.DataFrame({'Q': Q_H_H, 'Type': 'Homopolymer_vs_Homopolymer'})
    dataframes_list.append(df_H_H)
else:
    df_H_H = pd.DataFrame({'Q': [], 'Type': []})

if len(Q_M_F) > 0:
    df_M_F = pd.DataFrame({'Q': Q_M_F, 'Type': 'MiChroM_vs_DirectInversion'})
    dataframes_list.append(df_M_F)
else:
    df_M_F = pd.DataFrame({'Q': [], 'Type': []})

if len(Q_M_H) > 0:
    df_M_H = pd.DataFrame({'Q': Q_M_H, 'Type': 'MiChroM_vs_Homopolymer'})
    dataframes_list.append(df_M_H)
else:
    df_M_H = pd.DataFrame({'Q': [], 'Type': []})

if len(Q_F_H) > 0:
    df_F_H = pd.DataFrame({'Q': Q_F_H, 'Type': 'DirectInversion_vs_Homopolymer'})
    dataframes_list.append(df_F_H)
else:
    df_F_H = pd.DataFrame({'Q': [], 'Type': []})

In [ ]:
# Only concatenate non-empty DataFrames
dfs_to_concat = [df for df in [df_M_M, df_F_F, df_H_H, df_M_F, df_M_H, df_F_H] if len(df) > 0]
df_combined = pd.concat(dfs_to_concat, ignore_index=True) if len(dfs_to_concat) > 0 else pd.DataFrame({'Q': [], 'Type': []})
# df_combined = pd.concat([df_F_F,df_M_M, df_M_F, df_F_H, df_M_H,df_H_H], ignore_index=True)

In [ ]:
np.mean(Q_M_M),np.mean(Q_F_F),np.mean(Q_H_H)

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))

# for column in df1.columns:
#     sns.kdeplot(df1[column], label=column)

# plt.legend()
# plt.show()

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))

# for column in df2.columns:
#     sns.kdeplot(df2[column], label=column)

# plt.legend()
# plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.kdeplot(data=df_combined, x='Q', hue='Type', fill=True, legend=True)
plt.title('Distribution of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.show()


In [ ]:
selected_types = ['Homopolymer_vs_Homopolymer', 'MiChroM_vs_DirectInversion', 'MiChroM_vs_MiChroM','DirectInversion_vs_DirectInversion']
# selected_types = ['Homopolymer_vs_Homopolymer', 'MiChroM_vs_DirectInversion']
# df_filtered = df_combined[df_combined['Type'].isin(selected_types)]

df_filtered = df_combined

In [ ]:


epsilon = 0.001 # a small value


# min_val = df_filtered['Q'].quantile(0.0005) + epsilon
min_val = df_combined['Q'].min() - epsilon
max_val = df_combined['Q'].quantile(0.9995) #- 0.1 #epsilon

df_combined.loc[:, 'Normalized_Q'] = (df_combined['Q'] - min_val) / (max_val - min_val)


In [ ]:
print(len(df_combined[df_combined['Type'] == 'MiChroM_vs_MiChroM']['Normalized_Q']))
print(len(df_combined[df_combined['Type'] == 'DirectInversion_vs_DirectInversion']['Normalized_Q']))
print(len(df_combined[df_combined['Type'] == 'Homopolymer_vs_Homopolymer']['Normalized_Q']))
print(len(df_combined[df_combined['Type'] == 'MiChroM_vs_DirectInversion']['Normalized_Q']))
print(len(df_combined[df_combined['Type'] == 'MiChroM_vs_Homopolymer']['Normalized_Q']))
print(len(df_combined[df_combined['Type'] == 'DirectInversion_vs_Homopolymer']['Normalized_Q']))

def array_to_square(arr):
    """
    Convert a 1D array to a 2D square matrix.
    """
    arr_numpy = arr.to_numpy()  # Convert the Series to numpy array
    n = int(np.sqrt(len(arr_numpy)))
    if n*n != len(arr_numpy):
        raise ValueError("Input array length is not a perfect square")
    return arr_numpy.reshape(n, n)


In [ ]:
Q_M_M_matrix_combined = upper_triangle_to_square(df_combined[df_combined['Type'] == 'MiChroM_vs_MiChroM']['Normalized_Q'])
Q_F_F_matrix_combined = upper_triangle_to_square(df_combined[df_combined['Type'] == 'DirectInversion_vs_DirectInversion']['Normalized_Q'])
Q_M_F_matrix_combined = array_to_square(df_combined[df_combined['Type'] == 'MiChroM_vs_DirectInversion']['Normalized_Q'])
Q_H_H_matrix_combined = upper_triangle_to_square(df_combined[df_combined['Type'] == 'Homopolymer_vs_Homopolymer']['Normalized_Q'])
Q_M_H_matrix_combined = array_to_square(df_combined[df_combined['Type'] == 'MiChroM_vs_Homopolymer']['Normalized_Q'])
Q_F_H_matrix_combined = array_to_square(df_combined[df_combined['Type'] == 'DirectInversion_vs_Homopolymer']['Normalized_Q'])

In [ ]:
merged_matrix_combined = np.block([
    [1.2*Q_M_M_matrix_combined, 1.1*Q_M_F_matrix_combined, Q_M_H_matrix_combined],
    [1.1*Q_M_F_matrix_combined.T, Q_F_F_matrix_combined, Q_F_H_matrix_combined],
    [Q_M_H_matrix_combined.T, Q_F_H_matrix_combined.T, 1.05*Q_H_H_matrix_combined]
])


In [ ]:
merged_matrix_combined_C = coarsen(np.mean,merged_matrix_combined,{0: 1, 1: 1}, trim_excess=True)

In [ ]:
fig = plt.figure(figsize=(5,5),frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

vmin=0.0
vmax=1.0


# palette = cubehelix.classic_16.mpl_colormap
# palette = palettable.mycarta.LinearL_4.mpl_colormap
# palette = palettable.lightbartlein.diverging.BlueDarkRed18_9.mpl_colormap
# palette = palettable.scientific.sequential.Acton_3_r.mpl_colormap

palette = palettable.cmocean.sequential.Ice_3_r.mpl_colormap

cax = ax.imshow(merged_matrix_combined_C, interpolation='None', cmap=palette, vmin=vmin, vmax=vmax)

plt.colorbar(cax, ax=ax, fraction=0.046, pad=0.04)

# plt.savefig('/home/vinicius/Work/Mammoth/Simulations/Q_Matrix_color.pdf',dpi=600, pad_inches=0)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import palettable


# Extract key colors from the palettable colormap
palette = palettable.cmocean.sequential.Ice_3_r.mpl_colormap
start_color = palette(0.0)    # Color at the start of the colormap
middle_color = palette(0.4) # Color at the middle of the colormap
end_color = palette(0.99)      # Color at the end of the colormap

# Define custom color transitions
vmin = 0.25
vmax = 0.8
norm = mpl.colors.Normalize(vmin, vmax)
colors = [
    [norm(vmin), start_color],
    [norm(0.4), middle_color], # Transition starts at 0.4
    [norm(0.4), middle_color],    # Rapid transition after 0.4
    [norm(vmax), end_color]
]

# Create a custom colormap
custom_colormap = mpl.colors.LinearSegmentedColormap.from_list("custom_colormap", colors)

# Your plotting code
fig = plt.figure(figsize=(5,5), frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

# Use the new colormap
cax = ax.imshow(merged_matrix_combined_C, interpolation='bicubic', cmap=custom_colormap, vmin=vmin, vmax=vmax)

plt.colorbar(cax, ax=ax, fraction=0.046, pad=0.04)
# plt.savefig('/home/vinicius/Work/Mammoth/Simulations/Q_Matrix_color2.pdf', dpi=600, pad_inches=0)
plt.show()

In [ ]:
# epsilon = 0.0  # a small value
# min_val = df_filtered['Q'].min() + epsilon
# max_val = df_filtered['Q'].max() - epsilon

# df_filtered.loc[:, 'Normalized_Q'] = (df_filtered['Q'] - min_val) / (max_val - min_val)


epsilon = 0.001 # a small value
# type_specific_df_max = df_filtered[df_filtered['Type'] == 'MiChroM_vs_MiChroM']
# type_specific_df_min = df_filtered[df_filtered['Type'] == 'Homopolymer_vs_Homopolymer']
# min_val = #type_specific_df_min['Q'].min() + epsilon
# max_val = 0.65 #type_specific_df_max['Q'].max() - epsilon

# min_val = df_filtered['Q'].quantile(0.0005) + epsilon
min_val = df_filtered['Q'].min() - epsilon
max_val = df_filtered['Q'].quantile(0.9995) #- 0.1 #epsilon

df_filtered.loc[:, 'Normalized_Q'] = (df_filtered['Q'] - min_val) / (max_val - min_val)


In [ ]:
min_val,max_val

In [ ]:
# # Filter the data
# selected_types = ['MiChroM_vs_DirectInversion', 'Homopolymer_vs_Homopolymer']
# df_two_types = df_filtered[df_filtered['Type'].isin(selected_types)]

# # Set the default font size for various elements
# plt.rcParams['axes.labelsize'] = 30
# plt.rcParams['axes.titlesize'] = 30
# plt.rcParams['xtick.labelsize'] = 20
# plt.rcParams['ytick.labelsize'] = 20

# # Plot size
# plt.figure(figsize=(12, 8))

# # Colors
# colors = ['#808080','#007A1D']

# # Plot
# sns.kdeplot(data=df_two_types, x='Normalized_Q', hue='Type', fill=True, palette=colors, alpha=.75, linewidth=0, common_norm=False, legend=False, bw_adjust=1.5)

# custom_labels = ['MiChroM+Full', 'Homopolymer']
# plt.legend(labels=custom_labels, loc='upper left', frameon=False, fontsize=23)

# # Labels
# # plt.ylabel(r"Probability Density [PD$(Q_{\mathrm{norm}}$)]")
# # plt.xlabel(r'Similarity [$Q_{\mathrm{norm}}$]')

# plt.ylabel(r"Probability Density [PD(Q)]")
# plt.xlabel(r'Similarity [Q]')

# # Limit
# plt.xlim(0.0, 1.0)



# # Save and Show
# plt.savefig('/mnt/d/Mammoth/Data_Final/PQ_Qnorm_test.jpg', dpi=600)
# plt.show()


In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='seaborn')

In [ ]:
colors = palettable.cmocean.sequential.Speed_7_r.mpl_colors

In [ ]:
def hex_to_rgb(value):
    value = value.lstrip('#')
    length = len(value)
    return tuple(int(value[i:i+length//3], 16)/255.0 for i in range(0, length, length//3))

In [ ]:
colors = colors[1:-1]
# Add the gray color at the beginning
gray_rgb = hex_to_rgb("#808080")
colors.append(gray_rgb)

In [ ]:
colors

In [ ]:
# import matplotlib.patches as patches
# fig, ax = plt.subplots(1, figsize=(5, len(colors)))

# for idx, color in enumerate(colors):
#     ax.add_patch(patches.Rectangle((0, idx), 1, 1, facecolor=color))

# ax.set_xlim(0, 1)
# ax.set_ylim(0, len(colors))
# ax.axis('off')  # Hide the axes
# plt.show()

In [ ]:
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

order = ['DirectInversion_vs_DirectInversion','MiChroM_vs_MiChroM','MiChroM_vs_DirectInversion','DirectInversion_vs_Homopolymer','MiChroM_vs_Homopolymer','Homopolymer_vs_Homopolymer']
# colors = ['#291E8F', '#AD9C00','#007A1D','#808080']
color_dict = dict(zip(df_filtered['Type'].unique(), colors))
ordered_colors = [color_dict[item] for item in order]

# Modify the height for larger plots
height_value = 1.5
# g = sns.FacetGrid(df_filtered, row="Type", hue="Type", aspect=5, height=height_value, palette=ordered_colors, row_order=order)

g = sns.FacetGrid(df_filtered, row="Type", hue="Type", aspect=5, height=height_value, palette=colors, row_order=order)

# Draw the densities in a few steps
g.map(sns.kdeplot, "Normalized_Q",
      bw_adjust=1.5, clip_on=False,
      fill=True, alpha=0.75, linewidth=0,common_norm=False, legend=False)
g.map(sns.kdeplot, "Normalized_Q", clip_on=False, color="w", lw=2, bw_adjust=1.5)

# Define and use a simple function to label the plot in axes coordinates
# def label(x, color, label):
#     ax = plt.gca()
#     ax.text(0, .2, label, fontweight="normal", color=color, fontsize=23,
#             ha="left", va="bottom", transform=ax.transAxes)

# g.map(label, "Normalized_Q")

# Set the subplots to overlap
g.figure.subplots_adjust(hspace=-.5)

# Remove axes details that don't play well with overlap
g.set_titles("")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)

# g.set_xlabels(r'Similarity [$Q_{\mathrm{norm}}$]', fontsize=23)
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_fontsize(18)


# g.axes[2, 0].set_ylabel(r"Probability Density [PD$(Q_{\mathrm{norm}}$)]", fontsize=23)

# # Modify y-tick labels font size for the last axis
# for label in g.axes[3, 0].get_yticklabels():
#     label.set_fontsize(18)

# custom_labels = ['MiChroM+Full', 'Homopolymer']
# plt.legend( labels=custom_labels, loc='upper left')
# plt.ylabel(r"Probability Density [PD$(Q_{\mathrm{norm}}$)]")
# plt.xlabel(r'Similarity [$Q_{\mathrm{norm}}$]')
plt.xlim(0.0, 1.0)

for ax in g.axes.flat:
    ax.spines['bottom'].set_visible(True)
    ax.spines['bottom'].set_linewidth(1)

# plt.savefig('/mnt/d/Mammoth/Data_Final/PQ_Qnorm_ridge_3.eps',dpi=600)

plt.show()


In [ ]:
# a = sns.choose_colorbrewer_palette('q')

In [ ]:
# sns.choose_cubehelix_palette(as_cmap=True)

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 6))
# sns.violinplot(data=df_filtered, x='Type', y='Normalized_Q', palette='muted')
# plt.title('Normalized Distribution of Selected Data')
# plt.ylabel('Normalized Value')
# plt.show()


In [ ]:
# df = pd.DataFrame({
#     'MiChroM': Rg_M_M,
#     'Full': Rg_F_F,
#     'HP': Rg_H_H
# })

# df_long = df.melt(var_name='Type', value_name='Rg')

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))
# sns.kdeplot(data=df_long, x='Rg', hue='Type', fill=True)
# plt.title('Distribution of Rg for Different Types')
# plt.xlabel('Rg')
# plt.ylabel('Density')
# plt.show()


In [ ]:
# colors = {1: 'orange', 2: 'green', 3: 'red', 4: 'blue', 5: 'purple', 6: 'brown', 7: 'pink', 8: 'gray', 9: 'olive', 10: 'cyan'}
# c=[colors[i] for i in T ]

In [ ]:

# # initialize MDS with number of dimensions as 2
# mds = manifold.MDS(n_components=2, dissimilarity='precomputed')

# # fit and transform the similarity matrix
# X_transformed = mds.fit_transform(dissimilarity_matrix)

# # plot the transformed data
# plt.scatter(X_transformed[:, 0], X_transformed[:, 1],c=[colors[i] for i in T ])
# # plt.savefig('/mnt/d/DT40/Centromere-Modeling/Data_Gro/Figures/Dendogram_3D_Images/MDS_sim_chr5_1000_set1.jpg',dpi=300)
# # plt.scatter(X_transformed[:, 0], X_transformed[:, 1])

In [ ]:
import numpy as np

def kabsch(P, Q):
    """
    P and Q are N x 3 matrices.
    Returns the optimal rotation matrix R for aligning points in P to points in Q.
    """
    # Center the points about the origin
    centroid_P = np.mean(P, axis=0)
    centroid_Q = np.mean(Q, axis=0)
    
    P_centered = P - centroid_P
    Q_centered = Q - centroid_Q
    
    # Calculate the covariance matrix
    H = np.dot(P_centered.T, Q_centered)
    
    # Singular Value Decomposition
    U, S, Vt = np.linalg.svd(H)
    
    # Ensure a right-handed coordinate system
    d = (np.linalg.det(U) * np.linalg.det(Vt)) < 0.0
    
    if d:
        S[-1] = -S[-1]
        U[:, -1] = -U[:, -1]
    
    # Calculate the optimal rotation matrix
    R = np.dot(U, Vt)
    
    return R

In [ ]:
def kabsch_with_translation(P, Q):
    """
    P and Q are N x 3 matrices.
    Returns the translated and rotated version of Q that aligns with P.
    """
    centroid_P = np.mean(P, axis=0)
    centroid_Q = np.mean(Q, axis=0)
    
    P_centered = P - centroid_P
    Q_centered = Q - centroid_Q
    
    R = kabsch(P_centered, Q_centered)  # Assuming you've defined the kabsch function
    
    # Apply rotation
    Q_rotated = np.dot(Q_centered, R.T)
    
    # Translate the rotated Q to align centroids
    Q_aligned = Q_rotated + centroid_P
    
    return Q_aligned


In [ ]:
# Example
P = XYZ_MiChroM[0]
Q = XYZ_HP[10]
# Q = XYZ_MiChroM[1]


In [ ]:
R = kabsch(P, Q)
# Q_rotated = np.dot(Q, R.T)

In [ ]:
Q_aligned = kabsch_with_translation(P, Q)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(12, 6))

# Plot for averaged_structure
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(Q[:, 0], Q[:, 1], Q[:, 2], marker='o')
ax1.scatter(P[:, 0], P[:, 1], P[:, 2], marker='o',color='red')
ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax1.set_zlabel("Z")

# Plot for reduced_structure
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(Q_aligned[:, 0], Q_aligned[:, 1], Q_aligned[:, 2], marker='o')
ax2.scatter(P[:, 0], P[:, 1], P[:, 2], marker='o',color='red')
ax2.set_xlabel("X")
ax2.set_ylabel("Y")
ax2.set_zlabel("Z")

plt.show()


In [ ]:
def tm_score(P_aligned, Q_aligned, L_target):
    """
    P_aligned and Q_aligned are N x 3 matrices of aligned points.
    L_target is the length of the target protein or structure.
    """
    # Determine the scaling factor d0
    d0 = 1.24 * np.sqrt(L_target - 15)
    d0 = min(d0, 3.0)

    # Calculate the distance between each pair of aligned points
    distances = np.sqrt(np.sum((P_aligned - Q_aligned)**2, axis=1))

    # Compute the TM-score
    score = np.sum(1.0 / (1.0 + (distances / d0)**2)) / L_target

    return score

In [ ]:
tm_score(P, Q_aligned, len(P)),tm_score(P, Q, len(P))

In [ ]:
compute_Q(P, Q, 481, 0, sigma=1.0),compute_Q(P, Q_aligned, 481, 0, sigma=1.0)

In [ ]:
# coords1 = np.array([[1.2, 3.4, 1.5],[4.0, 2.8, 3.7],[1.2, 4.2, 4.3],[0.0, 1.0, 2.0]])

In [ ]:
# coords2 = np.array([[2.3, 7.4, 1.5],[4.0, 2.9, -1.7],[1.2, 4.2, 4.3]])

In [ ]:
# def best_alignment_and_tm_score(P, Q):
#     """
#     Finds the best alignment of a subset of Q to P using the TM-score, or vice versa.
#     This function can handle cases where either P or Q is shorter.
#     """
#     if len(P) > len(Q):
#         P, Q = Q, P  # Swap P and Q if P is longer
    
#     best_score = -np.inf
#     best_alignment = None

#     for i in range(len(Q) - len(P) + 1):
#         Q_subset = Q[i:i+len(P)]
        
#         Q_aligned = kabsch_with_translation(P, Q_subset)
        
#         score = tm_score(P, Q_aligned, len(P))
        
#         if score > best_score:
#             best_score = score
#             best_alignment = Q_aligned
            
#     return best_alignment, best_score

In [ ]:
# best_align, score = best_alignment_and_tm_score(coords2, coords1)

In [ ]:
# best_align, score

In [ ]:
# import numpy as np

# def kabsch(P, Q):
#     # Centroid calculation
#     centroid_P = np.mean(P, axis=0)
#     centroid_Q = np.mean(Q, axis=0)
    
#     P_centered = P - centroid_P
#     Q_centered = Q - centroid_Q

#     # Calculate covariance matrix
#     C = np.dot(P_centered.T, Q_centered)

#     # Singular value decomposition
#     V, _, W = np.linalg.svd(C)
#     if np.linalg.det(V) * np.linalg.det(W) < 0:
#         W[2, :] *= -1

#     # Calculate the rotation matrix
#     R = np.dot(V, W)

#     return R

# def kabsch_with_translation(P, Q):
#     centroid_P = np.mean(P, axis=0)
#     centroid_Q = np.mean(Q, axis=0)
    
#     P_centered = P - centroid_P
#     Q_centered = Q - centroid_Q
    
#     R = kabsch(P_centered, Q_centered)
    
#     # Apply rotation
#     Q_rotated = np.dot(Q_centered, R.T)
    
#     # Translate the rotated Q to align centroids
#     Q_aligned = Q_rotated + centroid_P
    
#     return Q_aligned

# def tm_score(P_aligned, Q_aligned, L_target):
#     if L_target < 15:
#         d0 = 1.0  # arbitrary small value, but you can adjust as needed
#     else:
#         d0 = 1.24 * np.sqrt(L_target - 15)
#     d0 = min(d0, 3.0)
#     distances = np.sqrt(np.sum((P_aligned - Q_aligned)**2, axis=1))
#     score = np.sum(1.0 / (1.0 + (distances / d0)**2)) / L_target
#     return score


# def best_alignment_and_tm_score(P, Q):
#     if len(P) > len(Q):
#         P, Q = Q, P  # Swap P and Q if P is longer
    
#     best_score = -np.inf
#     best_alignment = None

#     for i in range(len(Q) - len(P) + 1):
#         Q_subset = Q[i:i+len(P)]
        
#         Q_aligned = kabsch_with_translation(P, Q_subset)
        
#         score = tm_score(P, Q_aligned, len(P))
        
#         if score > best_score:
#             best_score = score
#             best_alignment = Q_aligned
            
#     return best_alignment, best_score

# # Your provided coordinates
# coords1 = np.array([[1.2, 3.4, 1.5], [4.0, 2.8, 3.7], [1.2, 4.2, 4.3], [0.0, 1.0, 2.0]])
# coords2 = np.array([[2.3, 7.4, 1.5], [4.0, 2.9, -1.7], [1.2, 4.2, 4.3]])

# best_align, score = best_alignment_and_tm_score(coords2, coords1)



In [ ]:
# best_align, score

In [ ]:
# import numpy as np

# def kabsch(P, Q):
#     centroid_P = np.mean(P, axis=0)
#     centroid_Q = np.mean(Q, axis=0)
    
#     P_centered = P - centroid_P
#     Q_centered = Q - centroid_Q

#     C = np.dot(P_centered.T, Q_centered)

#     V, _, W = np.linalg.svd(C)
#     if np.linalg.det(V) * np.linalg.det(W) < 0:
#         W[2, :] *= -1

#     R = np.dot(V, W)
#     return R

# def kabsch_with_translation(P, Q):
#     centroid_P = np.mean(P, axis=0)
#     centroid_Q = np.mean(Q, axis=0)
    
#     P_centered = P - centroid_P
#     Q_centered = Q - centroid_Q
    
#     R = kabsch(P_centered, Q_centered)
#     Q_rotated = np.dot(Q_centered, R.T)
#     Q_aligned = Q_rotated + centroid_P
    
#     return Q_aligned

# def tm_score(P_aligned, Q_aligned, L_target):
#     if L_target < 15:
#         d0 = 1.0
#     else:
#         d0 = 1.24 * np.sqrt(L_target - 15)
#     d0 = min(d0, 3.0)
    
#     distances = np.sqrt(np.sum((P_aligned - Q_aligned)**2, axis=1))
#     score = np.sum(1.0 / (1.0 + (distances / d0)**2)) / L_target
#     return score

# def best_alignment_and_tm_score(P, Q):
#     if len(P) > len(Q):
#         P, Q = Q, P
    
#     best_score = -np.inf
#     best_alignment = None

#     for i in range(len(Q) - len(P) + 1):
#         Q_subset = Q[i:i+len(P)]
#         Q_aligned = kabsch_with_translation(P, Q_subset)
        
#         score = tm_score(P, Q_aligned, len(P))
        
#         if score > best_score:
#             best_score = score
#             best_alignment = Q_aligned
            
#     return best_alignment, best_score

# coords1 = np.array([[1.2, 3.4, 1.5], [4.0, 2.8, 3.7], [1.2, 4.2, 4.3], [0.0, 1.0, 2.0]])
# coords2 = np.array([[2.3, 7.4, 1.5], [4.0, 2.9, -1.7], [1.2, 4.2, 4.3]])

# best_align, score = best_alignment_and_tm_score(coords1, coords2)

# best_align_output = best_align.tolist()
# score_output = score


In [ ]:
# sequence = []
# for i in range(1, 11):
#     sequence.append(i)
#     sequence.append('B')
# for i in range(11, 21):
#     sequence.append(i)
#     sequence.append('A')
# for i in range(21, 31):
#     sequence.append(i)
#     sequence.append('C')


In [ ]:
# indices = sequence[::2]
# patches = sequence[1::2]

In [ ]:
# polymer_structure = np.random.rand(30, 3)

In [ ]:
# len(indices),len(patches),np.shape(polymer_structure)

In [ ]:
# current_patch_type = patches[0]
# patch_indices = []

In [ ]:
# averaged_structure = np.zeros_like(polymer_structure)

In [ ]:
# for i, patch in enumerate(patches):
#     if patch == current_patch_type:
#         patch_indices.append(i)
#     else:
#         # When the patch changes, calculate the COM of the current patch
#         COM = np.mean(polymer_structure[patch_indices, :], axis=0)
        
#         for index in patch_indices:
#             averaged_structure[index, :] = COM
            
#         patch_indices = [i]
#         current_patch_type = patch

In [ ]:
# # Handle the last patch
# if patch_indices:
#     COM = np.mean(polymer_structure[patch_indices, :], axis=0)
#     for index in patch_indices:
#         averaged_structure[index, :] = COM

In [ ]:
# current_patch_type = patches[0]
# patch_indices = []

# # This list will store the new reduced polymer structure
# reduced_structure = []
# reduced_sequence = []
# for i, patch in enumerate(patches):
#     if patch == current_patch_type:
#         patch_indices.append(i)
#     if patch != current_patch_type or i == len(patches) - 1:  # Adding condition for the last element
#         # When the patch changes or we are at the end, calculate the COM of the current patch
#         COM = np.mean(polymer_structure[patch_indices, :], axis=0)
#         reduced_structure.append(COM)
#         reduced_sequence.append(current_patch_type)
        
#         patch_indices = [i]
#         current_patch_type = patch

# reduced_structure = np.array(reduced_structure)


In [ ]:
# reduced_structure

In [ ]:
# reduced_sequence

In [ ]:

# import numpy as np
# import matplotlib.pyplot as plt


# fig = plt.figure(figsize=(12, 6))

# # Plot for averaged_structure
# ax1 = fig.add_subplot(121, projection='3d')
# ax1.scatter(averaged_structure[:, 0], averaged_structure[:, 1], averaged_structure[:, 2], marker='o')
# ax1.set_title("Averaged Structure")
# ax1.set_xlabel("X")
# ax1.set_ylabel("Y")
# ax1.set_zlabel("Z")

# # Plot for reduced_structure
# ax2 = fig.add_subplot(122, projection='3d')
# ax2.scatter(reduced_structure[:, 0], reduced_structure[:, 1], reduced_structure[:, 2], marker='o')
# ax2.set_title("Reduced Structure")
# ax2.set_xlabel("X")
# ax2.set_ylabel("Y")
# ax2.set_zlabel("Z")

# plt.show()
